# [AI_8기] CH 3 LLM,RAG 활용 AI 챗봇 팀 프로젝트 12조

In [1]:
import requests
import os
import json
from openai import OpenAI
from bs4 import BeautifulSoup
import re
from typing import List, Dict
import time
import pandas as pd

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("GPT_API")
client = OpenAI()

In [3]:
url = "https://terms.naver.com/list.naver?cid=48156&categoryId=48156"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [4]:
# LLM 호출 함수 (OpenAI API 예시)
def food_name_from_title(title: str) -> str:

    try:
        # LLM 호출
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "주어진 데이터는 제목이야. 여기서 요리의 이름만 뽑아줘. 요리의 제목은 1개야."},
                {
                    "role": "user",
                    "content": title
                }
            ]
        )
        recipe_name = completion.choices[0].message.content
        return recipe_name
    
    except Exception as e:
        print(f"LLM 호출 에러: {e}")
        return "요리 이름 추출 실패"
    

In [5]:
def get_recipe_list(page : int) -> str:

    params = {"page": page, "view_type" : "sm"}
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        return response.text
        
    
    except requests.exceptions.RequestException as e:
        print(f"{page} 에러발생 {e}")
        return ""

In [6]:
def get_recipe_detail(url : str) -> Dict:
    
    recipe_data = {
        "요리재료" : [],
        "기본정보" : {},
        "조리순서" : []
    }
    
    # 페이지에서 내용 가져오기
    try :
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 요리 재료 추출
        ingredient_h4 = soup.find("h4", id="TABLE_OF_CONTENT2")
        if ingredient_h4:
            ingredient_text = ingredient_h4.find_next("p").text
            ingredients_clean = re.sub(r'· (주재료) : ', '', ingredient_text)
            ingredients_clean = ingredients_clean.replace("· 부재료 :" , ",")
            ingredients_clean = ingredients_clean.replace("\xa0" , " ")
            
            pattern = r'(?<!\d),(?!\d)'  # 쉼표 앞뒤가 숫자가 아닌 경우만 매칭
            ingreidents_list = [item.strip() for item in re.split(pattern, ingredients_clean)]
            
            for item in ingreidents_list:
                parts = item.rsplit(" ",1)
                if len(parts) ==2:
                    name, amount = parts
                    recipe_data['요리재료'].append({"재료":name.strip(), "용량":amount.strip()})
                
                
        
 
        # 기본정보 추출
        info_h4 = soup.find("h4", id="TABLE_OF_CONTENT3")
        if info_h4:  
            info_para = info_h4.find_next("p")
            if info_para:
                for strong_tag in info_para.find_all("strong"):
                    key = strong_tag.text.strip("·").strip(" :").strip()
                    value = strong_tag.next_sibling.strip() if strong_tag.next_sibling else ""
                    recipe_data["기본정보"][key] = value

        
        
        #요리과정 추출
        recipe_h4 = soup.find("h4",id="TABLE_OF_CONTENT4")
        recipe_list = []
        #과정 순회
        if recipe_h4:
            for tag in recipe_h4.find_all_next():
                if tag.name =="h3" or tag.name =="h4":
                    break
                if tag.name == "p" and tag.text.strip():
                    recipe_list.extend(tag.text.strip().split("\n"))
        recipe_data['조리순서'] = recipe_list
        return recipe_data
    
      
    except requests.exceptions.RequestException as e:
        print(f"기사 내용 요청 중 에러 발생: {e}")
        return {"error":"요청 실패"}

In [7]:
def parse_recipe_info(html: str) -> List[Dict]:
    recipes = []
    soup = BeautifulSoup(html,"html.parser")
    
    recipes_list = soup.find("ul", class_="content_list")
    if not recipes_list:
        return recipes
    
    new_recipes = recipes_list.find_all("li", recursive=False)
    print(f"{len(new_recipes)}개의 레시피를 찾았습니다.")
    
    
    for item in new_recipes:
        
        
        a_tags = item.find_all("a", href=True, onclick=True)
        a_tag = a_tags[2]
        
        if not a_tag:
            print("cant find a tag")
            continue
        
        info = item.find("div","info_area")
        title = info.find_all("a", href=True, onclick=True)[0].text.strip()
        
        
        food_name = food_name_from_title(title)
        
        print("==========================================")
        print(f"이 링크는 {food_name} 레시피의 링크 입니다.")


        url_elem = a_tag["href"] if a_tag else ""
        url = "https://terms.naver.com" + url_elem
        print(url)
        
        recipe = get_recipe_detail(url) if url else ""
        
                
        food_recipe = {

            "요리명" : food_name,
            "요리재료" : recipe["요리재료"],
            "기본정보" : recipe["기본정보"],
            "조리순서" : recipe["조리순서"]
        }
        
        recipes.append(food_recipe)
        
        time.sleep(1)
    
    
    return recipes

In [ ]:

def save_to_files(recipes: List[Dict], start_page: int, end_page: int, base_path: str = "./"):

    valid_recipes = [recipe for recipe in recipes if recipe.get("요리명") and recipe.get("요리재료") and recipe.get("기본정보") and recipe.get("조리순서")]
    
    if not valid_recipes:
        print(f"{start_page}~{end_page} 페이지에 유요한 레시피가 없어 저장을 건너 띕니다.")
    
    
    # DataFrame 생성
    df = pd.DataFrame(valid_recipes)
    
    
    file_name = f"food_recipes_{start_page}-{end_page}.json"
    
    
    # DataFrame을 JSON 파일로 저장
    json_path = os.path.join(base_path, file_name)
    df.to_json(json_path, force_ascii=False, orient='records', indent=4)
    print(f"JSON 파일 저장 완료: {json_path}")
    
    # 데이터 수집 정보 출력
    print("\n[데이터 수집 정보]")
    print(f"수집 페이지: {start_page}-{end_page} 페이지")
    print(f"수집 레시피: 총 {len(df)}건")
    
    # 데이터 미리보기
    print("\n[데이터 미리보기]")
    print(df.head())

In [9]:
def main():
    start_page = 1
    end_page = 68 # 수집할 마지막 페이지 번호
    page_unit = 100 # 한 번에 처리할 페이지 수
    download_folder = './food_recipe' #저장 폴더 이름
    os.makedirs(download_folder, exist_ok=True) #폴더 생성
    print("레시피 정보 수집 중...")

    
    # 10페이지씩 처리
    for start_idx in range(start_page, end_page + 1, page_unit):
        all_articles = []
        end_idx = min(start_idx + page_unit - 1, end_page)
        page_count = {end_idx} - {start_idx}
        
        print(f"\n=== {start_idx}~{end_idx} 페이지 수집 시작 ===")
        
        for page in range(start_idx, end_idx + 1):
            html = get_recipe_list(page)
            
            if html:
                page_articles = parse_recipe_info(html)
                
                #빈 데이터 확인
                valid_articles = [article for article in page_articles if article.get("요리명") and article.get("요리재료") and article.get("조리순서")]
                
                if valid_articles:
                    all_articles.extend(valid_articles)
                    print(f"페이지 {page}: {len(valid_articles)}개의 유효한 레시피 정보 수집 완료")
                else:
                    print(f"페이지 {page}: 유효한 레시피가 없어 건너띕니다.")
                    
            else:
                print(f"페이지 {page}: 데이터를 가져오기 못했습니다.")

        
        if not all_articles:
            print(f"{start_idx}~{end_idx} 페이지의 유효한 음식 레시피 정보가 없어 저장을 건너 띕니다.")
            continue
        
        print(f"\n{start_idx}~{end_idx} 페이지 : 총 {len(all_articles)}개의 유효한 레시피 정보를 수집했습니다.")
       
        
        try:
            save_to_files(all_articles, start_idx, end_idx, download_folder)
        
        except Exception as e:
            print(f"레시피 정보 저장 중 에러 발생: {e}")
        
        print(f"=== {start_idx}~{end_idx} 페이지 처리 완료 ===\n")
        
        # 다음 수집 전 잠시 대기
        time.sleep(0.1)

    print("\n모든 페이지 수집 완료!")

if __name__ == "__main__":
    main()

레시피 정보 수집 중...

=== 1~68 페이지 수집 시작 ===
15개의 레시피를 찾았습니다.
이 링크는 미역국 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=1988772&cid=48163&categoryId=48200
이 링크는 연근조림 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=1988751&cid=48164&categoryId=48203
이 링크는 늙은호박죽 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=1988548&cid=48161&categoryId=48197
이 링크는 배추김치 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=3390165&cid=48165&categoryId=48210
이 링크는 백김치 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=1988798&cid=48165&categoryId=48210
이 링크는 낙지볶음 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=3390177&cid=48164&categoryId=48204
이 링크는 도토리묵무침 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=3390183&cid=48164&categoryId=48205
이 링크는 고구마 맛탕 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=1988409&cid=48164&categoryId=48206
이 링크는 갈비찜 레시피의 링크 입니다.
https://terms.naver.com/entry.naver?docId=3390245&cid=48164&categoryId=48208
이 링크는 유자청 레시피의 링크 입니다.
https://te